<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/WordPiece_%D0%A2%D0%BE%D0%BA%D0%B5%D0%BD%D0%B8%D0%B7%D0%B0%D1%82%D0%BE%D1%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**WordPiece Токенизатор**  
WordPiece Токенизатор представляет собой элегантный гибридный подход, разработанный для нахождения оптимального баланса между токенизацией на уровне слов и символов. Он разбивает слова на более мелкие, но семантически осмысленные единицы, называемые подсловами (subwords). Например, слово "неизвестные" может быть токенизировано как "не", "##извест", "##ные". Префикс ## указывает, что данный подтокен является продолжением предыдущего токена, а не началом нового слова.  
Основная идея WordPiece: Построить словарь токенов (включающий символы, подслова и, при необходимости, целые слова) таким образом, чтобы он был достаточно компактным, но при этом позволял эффективно представлять как известные, так и неизвестные слова, сохраняя при этом их лингвистический смысл.

**1. Принцип Работы WordPiece: Пошаговое Объяснение**  
Алгоритм WordPiece, подобно BPE, является итеративным методом, но отличается критерием выбора пар для слияния. Вместо простой частоты встречаемости, WordPiece использует метрику, основанную на вероятности или правдоподобии.  
Возьмем наш пример текста для обучения токенизатора: "В училище учитель учит ученикам по новому учебнику."

**1.1. Шаг 1: Предварительная Обработка Текста и Создание Начального Словаря**  
Перед началом обучения текст подвергается предварительной обработке:  
 Приведение к нижнему регистру: Все буквы переводятся в строчные, чтобы "Учитель" и "учитель" считались одним и тем же словом.  
 Удаление знаков препинания: Знаки препинания (кроме тех, что являются частью слова, например, в сокращениях, но в нашем примере их нет) удаляются или отделяются.  
 Разделение на слова: Текст сначала разбивается на слова по пробелам.  
Обработанный текст: "в училище учитель учит ученикам по новому учебнику"  
Начальный словарь (V₀): Начинаем с самого простого: каждый уникальный символ в нашем обработанном тексте становится отдельным токеном в нашем словаре. Пробел ( ) также является важным токеном, так как он помогает модели понимать границы слов.  
Уникальные символы в тексте: 'в', ' ', 'у', 'ч', 'и', 'л', 'щ', 'е', 'т', 'р', 'ь', 'н', 'к', 'а', 'м', 'п', 'о', 'б', 'ю'.  
V₀ = {'в', ' ', 'у', 'ч', 'и', 'л', 'щ', 'е', 'т', 'р', 'ь', 'н', 'к', 'а', 'м', 'п', 'о', 'б', 'ю'} (если бы были другие символы).  
Теперь мы можем представить весь наш обработанный текст как последовательность этих начальных токенов. Например, слово "училище" будет представлено как ['у', 'ч', 'и', 'л', 'и', 'щ', 'е'].  
Полная последовательность токенов в корпусе (после предварительной обработки): ['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'л', 'ь', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'к', 'а', 'м', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']  
Общее количество токенов в корпусе: 50.

**1.2. Шаг 2: Итеративное Слияние (Обучение Словаря)**  
Это ядро алгоритма WordPiece. Мы будем итеративно (повторяющимися шагами) объединять пары соседних токенов (биграммы), чтобы создавать новые, более длинные токены. Цель — найти такие пары, которые, будучи объединенными, максимально увеличивают "вероятность" (или "правдоподобие") нашего обучающего текста.  

**Формула оценки слияния (Score):** Для каждой пары соседних токенов A и B (биграммы AB) мы вычисляем "оценку слияния" по следующей формуле:  
$$
\text{Score}(A, B) = \frac{f(A, B)}{f(A) \times f(B)}
$$  
Где:  
• $f(A, B)$ — это количество раз, когда биграмма AB (токен A, за которым сразу следует токен B) встречается в текущем токенизированном корпусе.  
• $f(A)$ — это количество раз, когда токен A встречается в текущем токенизированном корпусе.  
• $f(B)$ — это количество раз, когда токен B встречается в текущем токенизированном корпусе.  

Данная формула измеряет, насколько часто A и B встречаются **вместе**, по сравнению с тем, насколько часто они встречаются **по отдельности**. Высокий балл означает, что A и B **очень часто появляются вместе**, и их слияние в один токен будет "полезным" для модели, так как оно позволяет более компактно (с меньшим количеством токенов) представить текст. Это эквивалентно максимизации правдоподобия обучающих данных, если мы рассматриваем процесс токенизации как генерацию последовательности токенов.




**Пример итераций с нашим текстом:**

**Итерация 1: Поиск первой лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'л', 'ь', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'к', 'а', 'м', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']`

- **Подсчет частот отдельных токенов (униграмм) в текущем корпусе:**
  - $f(\text{' '}) = 8$
  - $f(\text{'у'}) = 5$
  - $f(\text{'ч'}) = 5$
  - $f(\text{'и'}) = 5$
  - $f(\text{'л'}) = 2$
  - $f(\text{'щ'}) = 1$
  - $f(\text{'е'}) = 4$
  - $f(\text{'т'}) = 2$
  - $f(\text{'ь'}) = 1$
  - $f(\text{'н'}) = 2$
  - $f(\text{'к'}) = 2$
  - $f(\text{'а'}) = 1$
  - $f(\text{'м'}) = 2$
  - $f(\text{'п'}) = 1$
  - $f(\text{'о'}) = 3$
  - $f(\text{'в'}) = 2$ (один раз в начале "в", один — в "новому")
  - $f(\text{'б'}) = 1$


- **Подсчет частот биграмм (пар соседних токенов):**
  - $f(\text{'у', 'ч'}) = 5$ — в "училище", "учитель", "учит" (дважды), "ученикам", "учебнику"
  - $f(\text{'ч', 'и'}) = 5$ — аналогично
  - $f(\text{'и', 'л'}) = 1$ — в "училище"
  - $f(\text{'л', 'и'}) = 1$ — в "училище"
  - $f(\text{'и', 'щ'}) = 1$ — в "училище"
  - $f(\text{'щ', 'е'}) = 1$ — в "училище"
  - $f(\text{'е', ' '}) = 2$ — после "училище", "учитель"
  - $f(\text{'и', 'т'}) = 2$ — в "учит" (дважды)
  - $f(\text{'т', 'е'}) = 1$ — в "учитель"
  - $f(\text{'е', 'л'}) = 1$ — в "учитель"
  - $f(\text{'л', 'ь'}) = 1$ — в "учитель"
  - $f(\text{'ч', 'е'}) = 2$ — в "ученикам", "учебнику"
  - $f(\text{'е', 'н'}) = 1$ — в "ученикам"
  - $f(\text{'н', 'и'}) = 2$ — в "ученикам", "учебнику"
  - $f(\text{'и', 'к'}) = 2$ — в "ученикам", "учебнику"
  - $f(\text{'к', 'а'}) = 1$ — в "ученикам"
  - $f(\text{'а', 'м'}) = 1$ — в "ученикам"
  - $f(\text{' ', 'п'}) = 1$ — перед "по"
  - $f(\text{'п', 'о'}) = 1$ — в "по"
  - $f(\text{'о', ' '}) = 1$ — после "по"
  - $f(\text{'н', 'о'}) = 2$ — в "новому", "учебнику"
  - $f(\text{'о', 'в'}) = 1$ — в "новому"
  - $f(\text{'в', 'о'}) = 1$ — в "новому"
  - $f(\text{'о', 'м'}) = 1$ — в "новому"
  - $f(\text{'м', 'у'}) = 1$ — в "новому"
  - $f(\text{'у', ' '}) = 4$ — после "в", "училище", "учитель", "новому"
  - $f(\text{'е', 'б'}) = 1$ — в "учебнику"
  - $f(\text{'б', 'н'}) = 1$ — в "учебнику"
  - $f(\text{'н', 'и'}) = 2$ — уже учтено
  - $f(\text{'и', 'к'}) = 2$ — уже учтено
  - $f(\text{'к', 'у'}) = 1$ — в "учебнику"
  - $f(\text{'в', ' '}) = 1$ — после слова "в"


- **Вычисление оценок слияния по формуле:**  
$$
  \text{Score}(A, B) = \frac{f(A, B)}{f(A) \times f(B)}
$$

  Примеры:
  - $\text{Score}(\text{'у', 'ч'}) = \frac{5}{5 \times 5} = \frac{5}{25} = 0.2$
  - $\text{Score}(\text{'ч', 'и'}) = \frac{5}{5 \times 5} = 0.2$
  - $\text{Score}(\text{'л', 'ь'}) = \frac{1}{2 \times 1} = \frac{1}{2} = 0.5$
  - $\text{Score}(\text{'к', 'а'}) = \frac{1}{2 \times 1} = 0.5$
  - $\text{Score}(\text{'а', 'м'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'б', 'н'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'п', 'о'}) = \frac{1}{1 \times 3} \approx 0.333$
  - $\text{Score}(\text{'н', 'о'}) = \frac{2}{2 \times 3} \approx 0.333$
  - $\text{Score}(\text{'щ', 'е'}) = \frac{1}{1 \times 4} = 0.25$
  - $\text{Score}(\text{'е', 'б'}) = \frac{1}{4 \times 1} = 0.25$

- **Выбор лучшей биграммы:**  
  Наивысшую оценку $\text{Score} = 0.5$ имеют биграммы: `'л', 'ь'`, `'к', 'а'`, `'а', 'м'`, `'б', 'н'`.  
  При равенстве оценок алгоритм может выбрать первую по порядку в лексикографическом или в порядке появления.  
  Предположим, выбирается первая появляющаяся такая пара — `'л', 'ь'` (в слове "учитель").

- **Слияние `'л', 'ь'`:**  
  Все вхождения последовательности `['л', 'ь']` заменяются на новый токен `'ль'`.  
  Например, слово "учитель" из `['у', 'ч', 'и', 'т', 'е', 'л', 'ь']` становится `['у', 'ч', 'и', 'т', 'е', 'ль']`.

- **Обновление словаря:**  
  Добавляем новый токен `'ль'` в словарь:  
$$
  V_1 = V_0 \cup \{\text{'ль'}\}
$$


**Итерация 2: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'к', 'а', 'м', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']`  
  (Токен `'ль'` уже заменил последовательность `'л', 'ь'` в слове "учитель".)  
  **Общее количество токенов: 49.**

- **Подсчет частот отдельных токенов (обновленные):**
  - $f(\text{'л'}) = 1$ — всё ещё встречается в "училище" (`'и', 'л', 'и'`)
  - $f(\text{'ь'}) = 0$ — больше не встречается как отдельный токен
  - $f(\text{'ль'}) = 1$ — новый токен
  - Остальные частоты остаются без изменений:
    - $f(\text{'у'}) = 5$, $f(\text{'ч'}) = 5$, $f(\text{'и'}) = 5$, $f(\text{'е'}) = 4$, $f(\text{'т'}) = 2$, $f(\text{'к'}) = 2$, $f(\text{'а'}) = 1$, $f(\text{'м'}) = 2$, $f(\text{' '}) = 8$ и т.д.

- **Подсчет частот биграмм (обновленные):**
  - Биграмма `'л', 'ь'` больше не встречается.
  - Появилась новая биграмма: `'е', 'ль'` (из "учитель") → $f(\text{'е', 'ль'}) = 1$
  - Остальные биграммы, не затронутые заменой, сохраняют свои частоты:
    - $f(\text{'у', 'ч'}) = 5$, $f(\text{'ч', 'и'}) = 5$, $f(\text{'к', 'а'}) = 1$, $f(\text{'а', 'м'}) = 1$, $f(\text{'б', 'н'}) = 1$ и т.д.

- **Вычисление оценок слияния:**
  Используем формулу:
$$
  \text{Score}(A, B) = \frac{f(A, B)}{f(A) \times f(B)}
$$

  Примеры:
  - $\text{Score}(\text{'е', 'ль'}) = \frac{1}{4 \times 1} = 0.25$
  - $\text{Score}(\text{'к', 'а'}) = \frac{1}{2 \times 1} = 0.5$
  - $\text{Score}(\text{'а', 'м'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'б', 'н'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'у', 'ч'}) = \frac{5}{5 \times 5} = 0.2$

- **Выбор лучшей биграммы:**  
  Наивысшую оценку $\text{Score} = 0.5$ имеют биграммы: `'к', 'а'`, `'а', 'м'`, `'б', 'н'`.  
  Выбираем первую по порядку появления — `'к', 'а'` (в слове "ученикам").

- **Слияние `'к', 'а'`:**  
  Все вхождения последовательности `['к', 'а']` заменяются на новый токен `'ка'`.  
  Например, слово "ученикам" из `['у', 'ч', 'е', 'н', 'и', 'к', 'а', 'м']` становится `['у', 'ч', 'е', 'н', 'и', 'ка', 'м']`.

- **Обновление словаря:**  
  Добавляем новый токен `'ка'` в словарь:  
$$
  V_2 = V_1 \cup \{\text{'ка'}\}
$$



**Итерация 3: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'ка', 'м', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']`  
  (Токены `'ль'` и `'ка'` уже вставлены.)  
  **Общее количество токенов: 48.**

- **Подсчет частот отдельных токенов (обновленные):**
  - $f(\text{'к'}) = 1$ — встречается в "учебнику"
  - $f(\text{'а'}) = 0$ — больше не встречается как отдельный токен
  - $f(\text{'ка'}) = 1$
  - $f(\text{'ам'}) = 1$ — пока нет, но биграмма `'а', 'м'` всё ещё может существовать
  - $f(\text{'а', 'м'}) = 1$ — в последовательности `['ка', 'м']` → `'ка', 'м'`, а не `'а', 'м'`  
    → Значит, биграмма `'а', 'м'` **больше не существует**, так как `'а'` исчез как отдельный токен

  Однако в слове "ученикам" теперь: `['ка', 'м']`, значит:
  - Биграмма: `'ка', 'м'`, а не `'а', 'м'`

  Следовательно:
  - $f(\text{'а', 'м'}) = 0$ → биграмма исчезла
  - $f(\text{'ка', 'м'}) = 1$ → новая биграмма

- **Подсчет частот биграмм:**
  - Биграммы `'к', 'а'` и `'а', 'м'` больше не встречаются.
  - Появилась новая биграмма: `'ка', 'м'` → $f = 1$
  - Остальные биграммы без изменений, например:
    - $f(\text{'б', 'н'}) = 1$
    - $f(\text{'у', 'ч'}) = 5$
    - $f(\text{'ч', 'и'}) = 5$

- **Вычисление оценок:**
  - $\text{Score}(\text{'ка', 'м'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'б', 'н'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'у', 'ч'}) = \frac{5}{5 \times 5} = 0.2$

- **Выбор лучшей биграммы:**  
  Наивысшую оценку $0.5$ имеют `'ка', 'м'` и `'б', 'н'`.  
  Выбираем `'ка', 'м'` как следующую по порядку.

- **Слияние `'ка', 'м'`:**  
  Заменяем последовательность `['ка', 'м']` на новый токен `'кам'`.  
  Слово "ученикам" из `['у', 'ч', 'е', 'н', 'и', 'ка', 'м']` становится `['у', 'ч', 'е', 'н', 'и', 'кам']`.

- **Обновление словаря:**  
  Добавляем новый токен `'кам'` в словарь:  
$$
  V_3 = V_2 \cup \{\text{'кам'}\}
$$



**Итерация 4: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']`  
  (Токены `'ль'`, `'ка'`, `'ам'` уже объединены в `'кам'`. Уточнение: `'ам'` как отдельный токен не был добавлен — слияние произошло напрямую в `'кам'`.)  
  **Общее количество токенов: 47.**

- **Подсчет частот отдельных токенов (обновленные):**
  - $f(\text{'а'}) = 0$ — больше не встречается
  - $f(\text{'м'}) = 1$ — встречается в "новому" (`'м', 'у'`)
  - $f(\text{'кам'}) = 1$
  - Остальные частоты без изменений.

- **Подсчет частот биграмм (обновленные):**
  - Биграмма `'и', 'кам'` появляется один раз (в "ученикам").
  - $f(\text{'и', 'кам'}) = 1$
  - Остальные биграммы, не затронутые слиянием, сохраняют свои значения:
    - $f(\text{'б', 'н'}) = 1$ — в "учебнику"
    - $f(\text{'у', 'ч'}) = 5$
    - $f(\text{'ч', 'и'}) = 5$

- **Вычисление оценок слияния:**
$$
  \text{Score}(A, B) = \frac{f(A, B)}{f(A) \times f(B)}
$$
  - $\text{Score}(\text{'и', 'кам'}) = \frac{1}{5 \times 1} = 0.2$
  - $\text{Score}(\text{'б', 'н'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'у', 'ч'}) = \frac{5}{5 \times 5} = 0.2$

- **Выбор лучшей биграммы:**  
  Наивысшую оценку $0.5$ имеет биграмма `'б', 'н'`.

- **Слияние `'б', 'н'`:**  
  Заменяем последовательность `['б', 'н']` на новый токен `'бн'`.  
  Слово "учебнику" из `['у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']` становится `['у', 'ч', 'е', 'бн', 'и', 'к', 'у']`.

- **Обновление словаря:**  
  Добавляем новый токен `'бн'` в словарь:  
$$
  V_4 = V_3 \cup \{\text{'бн'}\}
$$



**Итерация 5: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'бн', 'и', 'к', 'у']`  
  (Токены `'ль'`, `'кам'`, `'бн'` добавлены.)  
  **Общее количество токенов: 46.**

- **Подсчет частот отдельных токенов (обновленные):**
  - $f(\text{'б'}) = 0$
  - $f(\text{'н'}) = 2$ — встречается в "ученикам" (`'е', 'н'`) и "новому" (`'н', 'о'`)
  - $f(\text{'бн'}) = 1$


- **Подсчет частот биграмм:**
  - Появились новые биграммы:
    - $f(\text{'е', 'бн'}) = 1$ — в "учебнику"
    - $f(\text{'бн', 'и'}) = 1$ — в "учебнику"
  - $f(\text{'н', 'о'}) = 2$ — в "новому" и "учебнику"
  - $f(\text{'о', 'в'}) = 1$, $f(\text{'в', 'о'}) = 1$, $f(\text{'о', 'м'}) = 1$

- **Вычисление оценок:**
  - $\text{Score}(\text{'е', 'бн'}) = \frac{1}{4 \times 1} = 0.25$
  - $\text{Score}(\text{'бн', 'и'}) = \frac{1}{1 \times 5} = 0.2$
  - $\text{Score}(\text{'н', 'о'}) = \frac{2}{2 \times 3} \approx 0.333$
  - $\text{Score}(\text{'о', 'в'}) = \frac{1}{3 \times 2} \approx 0.167$
  - $\text{Score}(\text{'в', 'о'}) = \frac{1}{2 \times 3} \approx 0.167$

- **Выбор лучшей биграммы:**  
  Наивысшую оценку $\approx 0.333$ имеет `'н', 'о'`.

- **Слияние `'н', 'о'`:**  
  Заменяем все вхождения `['н', 'о']` на новый токен `'но'`.  
  Слово "новому" из `['н', 'о', 'в', 'о', 'м', 'у']` становится `['но', 'в', 'о', 'м', 'у']`.

- **Обновление словаря:**  
$$
  V_5 = V_4 \cup \{\text{'но'}\}
$$


**Итерация 6: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'но', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'бн', 'и', 'к', 'у']`  
  **Общее количество токенов: 45.**

- **Подсчет частот:**
  - $f(\text{'но'}) = 1$
  - $f(\text{'в'}) = 2$ — в "в" и "новому"
  - $f(\text{'о'}) = 2$ — в "по", "новому"

- **Частоты биграмм:**
  - $f(\text{'но', 'в'}) = 1$ — в "новому"

- **Оценка слияния:**
$$
  \text{Score}(\text{'но', 'в'}) = \frac{1}{1 \times 2} = 0.5
$$

- **Выбор лучшей биграммы:**  
  Среди текущих биграмм `'но', 'в'` имеет одну из высоких оценок.

- **Слияние `'но', 'в'`:**  
  Заменяем `['но', 'в']` на новый токен `'нов'`.  
  Слово "новому" становится `['нов', 'о', 'м', 'у']`.

- **Обновление словаря:**  
$$
  V_6 = V_5 \cup \{\text{'нов'}\}
$$


**Итерация 7: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'нов', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'бн', 'и', 'к', 'у']`  
  **Общее количество токенов: 44.**

- **Частоты:**
  - $f(\text{'нов'}) = 1$
  - $f(\text{'о'}) = 2$

- **Биграмма:**
  - $f(\text{'нов', 'о'}) = 1$

- **Оценка слияния:**
$$
  \text{Score}(\text{'нов', 'о'}) = \frac{1}{1 \times 2} = 0.5
$$

- **Слияние `'нов', 'о'`:**  
  Заменяем `['нов', 'о']` на `'ново'`.  
  Слово "новому" становится `['ново', 'м', 'у']`.

- **Обновление словаря:**  
$$
  V_7 = V_6 \cup \{\text{'ново'}\}
$$




**Итерация 8: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'ново', 'м', 'у', ' ', 'у', 'ч', 'е', 'бн', 'и', 'к', 'у']`  
  (Токены `'ль'`, `'кам'`, `'бн'`, `'но'`, `'нов'`, `'ново'` уже добавлены.)  
  **Общее количество токенов: 43.**

- **Подсчет частот отдельных токенов (обновленные):**
  - $f(\text{'ново'}) = 1$
  - $f(\text{'м'}) = 1$ — встречается в "новому"
  - $f(\text{'нов'}) = 0$ — больше не встречается как отдельный токен
  - Остальные частоты без изменений.

- **Подсчет частот биграмм:**
  - $f(\text{'ново', 'м'}) = 1$ — в слове "новому"

- **Оценка слияния:**
$$
  \text{Score}(\text{'ново', 'м'}) = \frac{f(\text{'ново', 'м'})}{f(\text{'ново'}) \times f(\text{'м'})} = \frac{1}{1 \times 1} = 1.0
$$

- **Выбор лучшей биграммы:**  
  Биграмма `'ново', 'м'` имеет высокую оценку (1.0).

- **Слияние `'ново', 'м'`:**  
  Заменяем последовательность `['ново', 'м']` на новый токен `'новом'`.  
  Слово "новому" из `['ново', 'м', 'у']` становится `['новом', 'у']`.

- **Обновление словаря:**  
  Добавляем новый токен `'новом'` в словарь:  
$$
  V_8 = V_7 \cup \{\text{'новом'}\}
$$



**Итерация 9: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'новом', 'у', ' ', 'у', 'ч', 'е', 'бн', 'и', 'к', 'у']`  
  (Токены `'новом'` и другие уже включены.)  
  **Общее количество токенов: 42.**

- **Подсчет частот отдельных токенов:**
  - $f(\text{'новом'}) = 1$
  - $f(\text{'у'}) = 5$ — встречается в "училище", "учитель", "учит", "ученикам", "учебнику"

- **Частота биграммы:**
  - $f(\text{'новом', 'у'}) = 1$

- **Оценка слияния:**
$$
  \text{Score}(\text{'новом', 'у'}) = \frac{1}{1 \times 5} = 0.2
$$



- **Выбор лучшей биграммы:**  
  На данном шаге среди всех доступных биграмм `'новом', 'у'` может быть одной из лучших, особенно если других высокочастотных пар не осталось.

- **Слияние `'новом', 'у'`:**  
  Заменяем `['новом', 'у']` на новый токен `'новому'`.  
  Слово "новому" теперь представлено как `['новому']`.

- **Обновление словаря:**  
  Добавляем новый токен `'новому'` в словарь:  
$$
  V_9 = V_8 \cup \{\text{'новому'}\}
$$



Мы продемонстрировали, как слово *"новому"* постепенно собиралось из отдельных символов через последовательные слияния:  
`'н'+'о'→'но'`, `'но'+'в'→'нов'`, `'нов'+'о'→'ново'`, `'ново'+'м'→'новом'`, `'новом'+'у'→'новому'`.  
Таким образом, WordPiece может формировать **целые слова** как отдельные токены, если они часто встречаются и слияние оправдано с точки зрения правдоподобия.

Этот процесс продолжается итеративно до достижения желаемого размера словаря. В реальных моделях, таких как BERT, он проходит **тысячи или десятки тысяч итераций** на огромных текстовых корпусах.

В конечном итоге, словарь содержит смесь:
- отдельных символов,
- подслов (например, `'уч'`, `'кам'`),
- целых слов (например, `'новому'`),  
что позволяет эффективно кодировать как частотные, так и редкие или неизвестные слова.



**2.3. Когда остановить итерацию? (Критерии остановки)**

Процесс итеративного слияния в WordPiece обычно останавливается при выполнении одного из следующих условий:

- **Достигнут заданный размер словаря:**  
  Это — наиболее распространённый критерий. Разработчик заранее определяет максимальное количество токенов в словаре (например, 30 000 для BERT, 50 000 для RoBERTa). Обучение завершается, как только словарь достигает этого предела.

- **Невозможно выполнить дальнейшие слияния:**  
  Это происходит, когда в корпусе больше нет соседних токенов, которые можно объединить — например, весь текст представлен минимальным числом токенов, или все возможные биграммы уже слиты.

- **Оценка лучшей биграммы падает ниже порога:**  
  Можно установить минимальное значение \(\text{Score}(A, B)\), ниже которого слияния считаются незначимыми. Если даже лучшая доступная биграмма имеет слишком низкую оценку, процесс останавливается, так как дальнейшие слияния вносят незначительный вклад в правдоподобие текста.




**Пример конечного словаря (Vfinal) после множества итераций:**

На основе последовательных итераций, наш словарь постепенно будет включать в себя следующие токены:  
`{'в', ' ', 'у', 'ч', 'и', 'л', 'щ', 'е', 'т', 'р', 'ь', 'н', 'к', 'а', 'м', 'п', 'о', 'б', 'ль', 'ка', 'ам', 'бн', 'но', 'нов', 'ново', 'новом', 'новому', 'уч', 'чи', 'ит', 'ищ', 'ще', 'е ', 'ни', 'ку', 'учи', 'учил', 'учили', 'училище', 'учитель', 'учит', 'уче', 'учени', 'ученик', 'ученикам', 'учеб', 'учебни', 'учебник', 'учебнику'}`.

- **Начальные символы:** Все уникальные символы из текста (например, `'в'`, `' '` , `'у'`, `'ч'` и т.д.) остаются в словаре — они необходимы для токенизации редких или неизвестных слов.
- **Итеративные слияния:** В процессе обучения добавляются составные токены: `'ль'`, `'ка'`, `'бн'`, `'но'`, `'нов'`, `'ново'`, `'новом'`, `'новому'` — как результат последовательных оптимальных слияний.
- **Более длинные подслова и целые слова:** При продолжении процесса часто встречающиеся фрагменты и целые слова, такие как `'учи'`, `'учил'`, `'учили'`, `'училище'`, `'учитель'`, `'учит'`, `'ученикам'`, `'учебник'`, `'учебнику'`, `'по'`, также становятся отдельными токенами.

> **Важное примечание о префиксах:**  
> В реальных реализациях WordPiece (например, в BERT) для обозначения продолжения слова используется префикс `##`. Например, слово *"играющий"* может быть разбито на `["игра", "##ющий"]`. Это позволяет модели различать, где начинается новое слово, а где — продолжается предыдущее.  
> В нашем упрощённом примере мы опустили `##` для наглядности, но в реальных системах эта система критически важна для корректной обработки морфологии и редких слов.



**3. Токенизация Нового Текста с Использованием Обученного Словаря**

После завершения обучения и формирования финального словаря, токенизатор может обрабатывать любой новый текст. Процесс токенизации **одного слова** выглядит следующим образом:

1. Ищем **самый длинный подтокен** из словаря, который совпадает с началом (префиксом) слова.
2. Если такой подтокен найден, добавляем его в выходную последовательность и повторяем процесс для оставшейся части слова.
3. Если часть слова не может быть разбита на известные подтокены, она либо:
   - разбивается на отдельные символы (если они есть в словаре),
   - либо заменяется специальным токеном `[UNK]` (Unknown), если ни один фрагмент не найден.

Этот алгоритм называется **жадным разбиением по наибольшему префиксу** (greedy longest-match-first).


**Пример токенизации с гипотетическим Vfinal:**  
Текст: `"в училище учитель учит ученикам по новому учебнику"`

- **Слово "в":**  
  → "в" есть в словаре → `['в']`

- **Слово "училище":**  
  → "училище" есть в словаре → `['училище']`

- **Слово "учитель":**  
  → "учитель" есть в словаре → `['учитель']`

- **Слово "учит":**  
  → "учит" есть в словаре → `['учит']`

- **Слово "ученикам":**  
  → "ученикам" есть в словаре → `['ученикам']`

- **Слово "по":**  
  → "по" есть в словаре → `['по']`

- **Слово "новому":**  
  → "новому" есть в словаре → `['новому']`

- **Слово "учебнику":**  
  → "учебнику" есть в словаре → `['учебнику']`


**Финальная токенизация (с пробелами как отдельными токенами):**  
`['в', ' ', 'училище', ' ', 'учитель', ' ', 'учит', ' ', 'ученикам', ' ', 'по', ' ', 'новому', ' ', 'учебнику']`

> ⚠️ Примечание: В большинстве современных реализаций пробелы **не используются как отдельные токены**. Вместо этого они либо удаляются, либо заменяются на `##` при внутрисловном слиянии. Однако в нашем упрощённом примере пробел сохраняется для наглядности.



**Случай неизвестного слова:**  
Если бы в тексте появилось слово *"преподавательница"*, которого нет в словаре, но есть подтокены:
- `"препода"` → да
- `"##ватель"` → да
- `"##ница"` → да

То оно было бы токенизировано как:  
`['препода', '##ватель', '##ница']`

Если же ни один подтокен не найден (например, из-за опечатки), слово может быть заменено на `[UNK]`.


**4. Масштабирование WordPiece на Многопредметный Корпус**

При обучении на большом корпусе, состоящем из множества предложений или документов, алгоритм WordPiece адаптируется, чтобы сохранить лингвистическую корректность и избежать артефактов.

**Ключевые аспекты масштабирования:**

- **Обработка по границам предложений:**  
  Каждое предложение обрабатывается **независимо**. Подсчёт частот биграмм и слияние токенов **не пересекают границы предложений**. Это предотвращает бессмысленные слияния между последним словом одного предложения и первым — другого.

- **Специальные разделительные токены:**  
  Часто используются специальные маркеры:
  - `<s>` — начало предложения
  - `</s>` — конец предложения  
  Эти токены помогают модели понимать структуру текста и не допускают слияния через границы.

- **Запрет межпредложных слияний:**  
  Это принципиально важно. Например, если одно предложение заканчивается словом, оканчивающимся на `"ок"`, а следующее начинается с `"но"`, WordPiece **не будет** объединять `"ок"` и `"но"` в `"окно"`, потому что они находятся в разных предложениях.  
  Это гарантирует, что все формируемые токены имеют **лингвистический смысл в контексте одного предложения**.

- **Работа с большим корпусом:**  
  На практике WordPiece обучается на **гигантских корпусах** (например, весь Wikipedia). Процесс включает:
  - предварительную токенизацию на уровне слов,
  - построение начального символьного словаря,
  - десятки тысяч итераций слияния,
  - остановку по достижению целевого размера словаря (например, 30 000 токенов).



### **Пример: Обучение WordPiece на корпусе предложений**

Рассмотрим следующий корпус из трёх предложений:

1. `"Ученик учится в школе, а учитель учит ученика."`  
2. `"Новый студент изучает грамматику."`  
3. `"Учитель и студент читают книги."`



#### **Начальная настройка**

Перед обучением добавим специальные токены для обозначения границ предложений:  
- `<s>` — начало предложения  
- `</s>` — конец предложения

Также проведём предварительную обработку:
- Приведение к нижнему регистру
- Замена пробелов на символ `_` (для наглядности — в реальных реализациях используется пробел или `##`)
- Разделение на символы

**Токенизированный корпус (на начальном символьном уровне):**

```
<s> у _ ч _ е _ н _ и _ к _ _ у _ ч _ и _ т _ с _ я _ _ в _ _ ш _ к _ о _ л _ е _ , _ _ а _ _ у _ ч _ и _ т _ е _ л _ ь _ _ у _ ч _ и _ т _ _ у _ ч _ е _ н _ и _ к _ а _ . </s>
<s> _ н _ о _ в _ ы _ й _ _ с _ т _ у _ д _ е _ н _ т _ _ и _ з _ у _ ч _ а _ е _ т _ _ г _ р _ а _ м _ м _ а _ т _ и _ к _ у _ . </s>
<s> _ у _ ч _ и _ т _ е _ л _ ь _ _ и _ _ с _ т _ у _ д _ е _ н _ т _ _ ч _ и _ т _ а _ ю _ т _ _ к _ н _ и _ г _ и _ . </s>
```

> ⚠️ Примечание: В реальных реализациях (например, BERT) пробелы не заменяются на `_`, а обрабатываются как отдельные символы. Здесь `_` используется для наглядности.



#### **Итерации WordPiece на корпусе**

При обучении частоты биграмм считаются **внутри каждого предложения отдельно**, чтобы избежать слияний через границы. Затем частоты суммируются по всему корпусу.

**Пример подсчёта биграмм (гипотетический):**

- **Из предложения 1:**  
  - `('у', 'ч')`: 5  
  - `('и', 'к')`: 2  
  - `('_', 'у')`: 3  
  - `('а', '_')`: 2  
  - `('т', 'е')`: 1  

- **Из предложения 2:**  
  - `('н', 'о')`: 1  
  - `('с', 'т')`: 1  
  - `('т', 'у')`: 1  
  - `('г', 'р')`: 1  

- **Из предложения 3:**  
  - `('у', 'ч')`: 1  
  - `('т', 'е')`: 2  
  - `('ч', 'и')`: 1  
  - `('и', '_')`: 1  
  - `('к', 'н')`: 1  

**Суммарные частоты по корпусу:**
- `('у', 'ч')`: 5 + 1 = **6**
- `('_', 'у')`: 3 + 1 = **4**
- `('т', 'е')`: 1 + 2 = **3**
- `('и', 'к')`: 2
- `('с', 'т')`: 1 + 1 = **2**


#### **Решение об объединении**

На первой итерации пара `('у', 'ч')` имеет наивысшую частоту (6). Согласно формуле WordPiece:
$$
\text{Score}(A, B) = \frac{f(A, B)}{f(A) \times f(B)}
$$
она также, скорее всего, будет иметь высокую оценку, особенно если `у` и `ч` встречаются вместе значительно чаще, чем можно было бы ожидать случайно.

Мы объединяем `у` и `ч` в новый токен **`уч`**.

**Обновление корпуса:**  
Во всех предложениях заменяем последовательности `['у', 'ч']` на `['уч']`. Например:
- Было: `... _ у _ ч _ и _ т ...`  
- Стало: `... _ уч _ и _ т ...`

Процесс повторяется: пересчитываются частоты, выбирается следующая лучшая биграмма, выполняется слияние — и так до достижения целевого размера словаря.


#### **Почему этот подход важен**

- **Осмысленные токены:**  
  Поскольку слияния не пересекают границы предложений, исключаются артефакты вроде объединения конца одного предложения с началом другого (например, `"а_н"` из `"а."</s><s>н"`). Это гарантирует, что формируемые токены лингвистически осмысленны.

- **Целостность предложений:**  
  Сохранение границ помогает модели учитывать синтаксическую и семантическую структуру предложений — критично для задач, таких как машинный перевод, классификация или генерация.

- **Масштабируемость:**  
  Подход позволяет обрабатывать большие или потоковые корпусы, где не все данные доступны одновременно. Это делает WordPiece пригодным для промышленного использования.

- **Последовательная токенизация:**  
  Одинаковые слова в одинаковом контексте будут токенизированы одинаково, что обеспечивает стабильность и воспроизводимость.


### **5. Преимущества и недостатки WordPiece**

Хотя WordPiece широко используется (в BERT, DistilBERT и других моделях), у него есть как сильные стороны, так и ограничения.



#### **5.1. Преимущества WordPiece**

- **Эффективная обработка OOV-слов (Out-of-Vocabulary):**  
  WordPiece способен разбивать неизвестные слова на знакомые подслова. Например, *"программистом"* → `["программ", "##ист", "##ом"]`, даже если целое слово отсутствует в словаре. Это обеспечивает частичное понимание и устойчивость к лексической вариативности.

- **Контроль размера словаря:**  
  Размер словаря задаётся заранее (например, 30 000). Алгоритм итеративно выбирает наиболее "полезные" слияния, максимизируя правдоподобие текста. Это позволяет балансировать между компактностью и покрытием.

- **Гибридный подход:**  
  WordPiece сочетает преимущества символьной и словесной токенизации:
  - Частые слова (например, `"учитель"`) становятся одним токеном.
  - Редкие или сложные слова разбиваются на подслова.
  - Все входные данные могут быть представлены — нет "неизвестных" символов.

- **Универсальность (в варианте Byte-level WordPiece):**  
  Байтовый WordPiece (используется в моделях вроде RoBERTa) работает на уровне байтов, а не символов Unicode. Это делает его **инвариантным к кодировке**, способным обрабатывать любой текст (включая эмодзи, редкие символы), и **гарантирует отсутствие [UNK]**.



#### **5.2. Недостатки WordPiece**

- **Жадный и детерминированный алгоритм:**  
  На каждом шаге выбирается пара с максимальной оценкой. Такой подход не учитывает долгосрочных последствий и может привести к субоптимальной сегментации с точки зрения морфологии или семантики.

- **Отсутствие лингвистических знаний:**  
  WordPiece — статистический алгоритм. Он не "понимает" морфемы. Например, может разбить `"бежать"` на `["беж", "##ать"]`, хотя лингвистически более корректно — `["бег", "##ать"]`. Это снижает интерпретируемость и иногда — эффективность.

- **Вычислительная сложность:**  
  На ранних этапах обучения (при большом количестве символьных токенов) подсчёт частот всех биграмм требует значительных ресурсов. Для больших корпусов это может быть медленным.

- **Зависимость от начальной токенизации:**  
  Если перед WordPiece используется разбиение по словам, то ошибки на этом этапе (например, неудачная обработка знаков препинания) могут повлиять на итоговый словарь и сегментацию.

- **Неоднозначность сегментации:**  
  Одно и то же слово может быть разбито по-разному в зависимости от истории слияний. Например, `"ученик"` может стать `["уч", "##еник"]` или `["учени", "##к"]` — в зависимости от того, какие пары были выбраны ранее. Хотя на практике это редко вызывает проблемы, теоретически снижает согласованность.


### **Заключение**

WordPiece стал **краеугольным камнем** современных NLP-моделей. Его способность эффективно справляться с неизвестными словами, контролировать размер словаря и сохранять баланс между символами и словами сделала его выбором по умолчанию для таких моделей, как **BERT** и **DistilBERT**.

Однако его ограничения — жадность, отсутствие морфологического понимания — стимулировали разработку альтернатив:
- **SentencePiece** — обучается без предварительной токенизации по словам.
- **Unigram Language Model** — использует вероятностную модель для сегментации.
- **BoundlessBPE / BBPE** — улучшает морфологическую целостность.

Эта эволюция показывает: **нет универсального идеального токенизатора**. Выбор зависит от задачи, языка, требований к размеру модели и компромисса между эффективностью, точностью и универсальностью.


In [ ]:
import collections

class WordPieceTokenizer:
    def __init__(self):
        self.vocab = set()
        self.merges = []

    def preprocess_text(self, text):
        """
        Предварительная обработка текста: приведение к нижнему регистру и разбиение на слова.
        """
        # Удаляем знаки препинания и приводим к нижнему регистру
        processed_text = text.lower().replace('...', '').replace('.', '')
        # Разбиваем на слова по пробелам
        words = processed_text.split(' ')
        # Добавляем пробелы как отдельные токены между словами
        initial_tokens = []
        for i, word in enumerate(words):
            if word: # Убедимся, что слово не пустое
                initial_tokens.extend(list(word))
            if i < len(words) - 1:
                initial_tokens.append(' ') # Добавляем пробел между словами
        return initial_tokens, words # Возвращаем начальные токены и список слов для удобства

    def calculate_frequencies(self, tokens):
        """
        Подсчет частот отдельных токенов (униграмм) и биграмм в текущем корпусе.
        """
        unigram_freq = collections.defaultdict(int)
        bigram_freq = collections.defaultdict(int)

        for i in range(len(tokens)):
            unigram_freq[tokens[i]] += 1
            if i < len(tokens) - 1:
                bigram_freq[(tokens[i], tokens[i+1])] += 1
        return unigram_freq, bigram_freq

    def calculate_score(self, unigram_freq, bigram_freq):
        """
        Вычисление оценки слияния для всех возможных биграмм.
        Score(A, B) = frequency(AB) / (frequency(A) * frequency(B))
        """
        scores = {}
        for (token_a, token_b), freq_ab in bigram_freq.items():
            freq_a = unigram_freq[token_a]
            freq_b = unigram_freq[token_b]
            if freq_a > 0 and freq_b > 0: # Избегаем деления на ноль
                score = freq_ab / (freq_a * freq_b)
                scores[(token_a, token_b)] = score
        return scores

    def merge_tokens(self, tokens, best_bigram):
        """
        Слияние лучшей биграммы в корпусе.
        """
        merged_token = "".join(best_bigram)
        new_tokens = []
        i = 0
        while i < len(tokens):
            if i + 1 < len(tokens) and (tokens[i], tokens[i+1]) == best_bigram:
                new_tokens.append(merged_token)
                i += 2 # Пропускаем оба слитых токена
            else:
                new_tokens.append(tokens[i])
                i += 1
        return new_tokens

    def train(self, text, target_vocab_size=None, num_merges=None):
        """
        Обучение WordPiece токенизатора.
        Итеративно сливает токены до достижения целевого размера словаря
        или заданного количества слияний.
        """
        print(f"Исходный текст: '{text}'")
        current_tokens, _ = self.preprocess_text(text)

        # Начальный словарь состоит из всех уникальных символов
        self.vocab = set(current_tokens)
        print(f"\nШаг 1: Начальный словарь (V0) содержит {len(self.vocab)} токенов.")
        print(f"Начальный корпус: {current_tokens}")

        merges_count = 0
        while True:
            unigram_freq, bigram_freq = self.calculate_frequencies(current_tokens)
            scores = self.calculate_score(unigram_freq, bigram_freq)

            if not scores:
                print("\nНет больше биграмм для слияния. Остановка.")
                break

            # Находим биграмму с наивысшей оценкой
            best_bigram = max(scores, key=scores.get)
            best_score = scores[best_bigram]

            merged_token = "".join(best_bigram)

            # Критерий остановки: если новый токен уже в словаре
            if merged_token in self.vocab:
                # Если лучший токен уже существует, удалим его из scores, чтобы найти следующий лучший
                del scores[best_bigram]
                continue # Продолжаем поиск

            # Выводим информацию о текущем слиянии
            print(f"\nИтерация {merges_count + 1}:")
            print(f"  Лучшая биграмма для слияния: '{best_bigram[0]}' + '{best_bigram[1]}' -> '{merged_token}' (Score: {best_score:.4f})")

            # Выполняем слияние в корпусе
            current_tokens = self.merge_tokens(current_tokens, best_bigram)

            # Добавляем новый токен в словарь и сохраняем слияние
            self.vocab.add(merged_token)
            self.merges.append(best_bigram)
            merges_count += 1

            print(f"  Корпус после слияния: {current_tokens}")
            print(f"  Текущий размер словаря: {len(self.vocab)}")

            # Критерии остановки
            if target_vocab_size is not None and len(self.vocab) >= target_vocab_size:
                print(f"\nДостигнут целевой размер словаря ({target_vocab_size}). Остановка.")
                break
            if num_merges is not None and merges_count >= num_merges:
                print(f"\nДостигнуто заданное количество слияний ({num_merges}). Остановка.")
                break

        print(f"\nОбучение завершено. Итоговый словарь содержит {len(self.vocab)} токенов.")
        return sorted(list(self.vocab))

    def tokenize(self, text):
        """
        Токенизация нового текста с использованием обученного словаря WordPiece.
        """
        if not hasattr(self, 'vocab') or len(self.vocab) == 0:
            raise ValueError("Токенизатор не обучен. Сначала вызовите метод train().")

        print(f"\n--- Токенизация нового текста ---")
        print(f"Текст для токенизации: '{text}'")

        # Предварительная обработка текста для токенизации
        processed_text = text.lower().replace('...', '').replace('.', '')
        words = processed_text.split(' ')

        final_tokens = []
        for word in words:
            if not word: # Пропускаем пустые строки, если они возникли из-за множественных пробелов
                continue

            word_tokens = []
            remaining_word = word

            while remaining_word:
                found_match = False
                # Ищем самый длинный подтокен из словаря, который является префиксом оставшейся части слова
                for i in range(len(remaining_word), 0, -1):
                    subword = remaining_word[:i]

                    if subword in self.vocab:
                        word_tokens.append(subword)
                        remaining_word = remaining_word[i:]
                        found_match = True
                        break

                if not found_match:
                    # Если не удалось найти соответствующий токен, разбиваем на символы
                    # или используем токен [UNK]
                    if remaining_word[0] in self.vocab: # Если символ есть в словаре
                        word_tokens.append(remaining_word[0])
                    else: # Если символ даже не в начальном словаре
                        word_tokens.append('[UNK]')
                    remaining_word = remaining_word[1:]

            final_tokens.extend(word_tokens)
            final_tokens.append(' ') # Добавляем пробел между токенизированными словами

        # Удаляем последний пробел, если он есть
        if final_tokens and final_tokens[-1] == ' ':
            final_tokens.pop()

        print(f"Токенизированный текст: {final_tokens}")
        return final_tokens

    def get_vocab(self):
        """Возвращает текущий словарь токенов."""
        return sorted(list(self.vocab))

    def get_merges(self):
        """Возвращает список выполненных слияний."""
        return self.merges

# --- Пример использования ---
if __name__ == "__main__":
    text_to_train = "В училище учитель учит ученикам по новому учебнику"

    # Создаем экземпляр токенизатора
    tokenizer = WordPieceTokenizer()

    # Обучаем токенизатор
    final_vocab = tokenizer.train(text_to_train, num_merges=50)

    print(f"\nИтоговый словарь WordPiece: {final_vocab}")

    # Токенизируем тот же текст
    tokenizer.tokenize(text_to_train)

    # Пример токенизации нового слова
    new_text = "учительница"
    # Добавим '##ница' в словарь для демонстрации
    if 'ница' not in final_vocab:
        tokenizer.vocab.add('ница')
        tokenizer.vocab.add('##ница')
        final_vocab = tokenizer.get_vocab()

    print(f"\nИтоговый словарь WordPiece (с 'ница' для демонстрации): {final_vocab}")
    tokenizer.tokenize(new_text)

    # Демонстрация с упрощенным словарем
    simplified_tokenizer = WordPieceTokenizer()
    simplified_tokenizer.vocab = set(['у', 'ч', 'и', 'т', 'е', 'л', 'ь', 'н', 'ц', 'а', 'учитель', '##ница', '##тель', '##а'])
    print(f"\n--- Демонстрация токенизации подслов с упрощенным словарем ---")
    simplified_tokenizer.tokenize("учительница")